In [ ]:
import os
import glob
import json

directory = 'datasets/국립국어원 신문 말뭉치(버전 2.0)'

json_files = glob.glob(os.path.join(directory, '**', '*.json'), recursive=True)
print(len(json_files))

In [ ]:
import sqlite3

connect = sqlite3.connect('nikl.db')
cursor = connect.cursor()

count = 0
for i, file in enumerate(json_files):
    print(i, file)
    with open(file, 'r', encoding='utf-8') as f:
        data_file = json.load(f)
    for document in data_file['document']:
        for paragraph in document['paragraph']:
            count += 1
            cursor.execute("INSERT INTO data VALUES(?, ?)",(count, paragraph['form']))

connect.commit()

# uni

In [ ]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline

n = 1
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

In [ ]:
from nltk.lm import MLE
uni_model = MLE(n)

In [ ]:
uni_model.fit(train_data, padded_sents)
print(uni_model.vocab)

In [ ]:
import dill as pickle

with open('uni_model.pkl', 'wb') as f:
    pickle.dump(uni_model, f)

# Bi

In [ ]:
from nltk.lm import MLE
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
import dill as pickle

n = 2
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
bi_model = MLE(n)
bi_model.fit(train_data, padded_sents)
with open('bi_model.pkl', 'wb') as f:
    pickle.dump(bi_model, f)

# Tri

In [ ]:
import pickle
from collections import defaultdict
import sqlite3
from collections import defaultdict
import re
from tqdm import tqdm

def calculate_trigram_probabilities(sentence, bigram_counts, trigram_counts):
    """
    Calculate trigram probabilities for a given sentence using loaded bigram and trigram counts.
    """
    # 토큰화된 문장 준비
    words = sentence.split()
    trigram_probabilities = {}

    for i in range(len(words) - 2):
        trigram = (words[i], words[i + 1], words[i + 2])
        bigram = (words[i], words[i + 1])

        # 바이그램과 트라이그램 빈도를 사용하여 확률 계산
        if trigram in trigram_counts and bigram in bigram_counts:
            trigram_probabilities[trigram] = trigram_counts[trigram] / bigram_counts[bigram]
        else:
            trigram_probabilities[trigram] = 0

    return trigram_probabilities



def calculate_ngram_counts(tokenized_text):
    """
    Calculate bigram and trigram counts from tokenized text.
    """
    trigram_counts = defaultdict(int)
    bigram_counts = defaultdict(int)

    for sentence in tqdm(tokenized_text, desc="Processing sentences"):
        for i in range(len(sentence) - 2):
            bigram = (sentence[i], sentence[i + 1])
            trigram = (sentence[i], sentence[i + 1], sentence[i + 2])
            bigram_counts[bigram] += 1
            trigram_counts[trigram] += 1

    # Remove trigrams that occur only once
    trigram_counts = {trigram: count for trigram, count in trigram_counts.items() if count > 1}

    return bigram_counts, trigram_counts

def save_ngram_counts(bigram_counts, trigram_counts, filename):
    """
    Save bigram and trigram counts to a .pkl file.
    """
    with open(filename, 'wb') as f:
        pickle.dump((bigram_counts, trigram_counts), f)

def load_ngram_counts(filename):
    """
    Load bigram and trigram counts from a .pkl file.
    """
    with open(filename, 'rb') as f:
        bigram_counts, trigram_counts = pickle.load(f)
    return bigram_counts, trigram_counts

# 예시 데이터베이스 경로 (이 경로는 변경해야 함)
db_path = 'nikl.db'

connect = sqlite3.connect(db_path)
cursor = connect.cursor()
tokenized_text = [list(map(str, sent[0].split())) for sent in cursor.execute('SELECT input FROM data')]
cursor.close()
connect.close()


# Example usage:
# tokenized_text = [...]  # This should be your actual tokenized text data
bigram_counts, trigram_counts = calculate_ngram_counts(tokenized_text)
save_ngram_counts(bigram_counts, trigram_counts, 'tri_model.pkl')
probs = calculate_trigram_probabilities('나는 학교에 간다', bigram_counts, trigram_counts)
probs  # 문장의 트라이그램 확률 출력
# To load the counts later:
# bigram_counts, trigram_counts = load_ngram_counts('ngram_counts.pkl')
